In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [40]:
best_model_path = "models/nf_new2.pth" ##3 best so far
target_col = 'No Finding'

In [24]:
class NoFindingRefiner(nn.Module):
    def __init__(self, input_dim=6):  # 1 from image model + 5 pathology scores
        super(NoFindingRefiner, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3), #was 0.5
            nn.Linear(64, 1),
            nn.Sigmoid()  # Because your scores range from 0 to 1
        )

    def forward(self, x):
        return self.model(x).squeeze(1)


In [41]:
class NoFindingRefiner2(nn.Module):
    def __init__(self, input_dim=6):  # 1 from image model + 5 pathology scores
        super(NoFindingRefiner2, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),  #0.3 for 3
            nn.Linear(32, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Dropout(0.3),  #0.3 for 3
            nn.Linear(20, 16),
            nn.ReLU(),
            nn.Dropout(0.3),  #0.3 for 3
            nn.Linear(16, 1),
            nn.Sigmoid()  # Because your scores range from 0 to 1
        )

    def forward(self, x):
        return self.model(x).squeeze(1)


In [46]:
class RefinementDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.features = ['NF_image', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                         'Pleural Effusion', 'Lung Opacity', 'Fracture']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x = torch.tensor(row[self.features].values, dtype=torch.float32)
        y = torch.tensor(row['No Finding'], dtype=torch.float32)
        return x, y

In [ ]:
features = ['NF_image', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                         'Pleural Effusion', 'Lung Opacity', 'Fracture']

In [47]:
df = pd.read_csv("train_predictions.csv")
df.rename(columns={'No Finding': 'NF_image'}, inplace=True)

df_ref = pd.read_csv('train2023.csv')
df_ref= df_ref.dropna(subset=['No Finding']).copy()
df_ref['No Finding'] = (df_ref['No Finding'] + 1) / 2

df_ref = df_ref.rename(columns={'Unnamed: 0': 'Id'})
df = df.merge(df_ref[['Id', 'No Finding']], on='Id', suffixes=('', '_true'))

In [48]:
print(df.head())

       Id  NF_image  Enlarged Cardiomediastinum  Cardiomegaly  Lung Opacity  \
0       0  0.920047                    0.017606     -0.720972     -0.489520   
1  100002 -0.998458                   -0.079644     -0.565395      0.905459   
2  100007 -0.999137                    0.091578     -0.878006      0.999590   
3  100009 -0.944439                   -0.529684     -0.929336      0.976516   
4  100017 -0.997833                    0.479632      0.831904      0.997616   

   Pneumonia  Pleural Effusion  Pleural Other  Fracture  Support Devices  \
0   0.031183         -0.817579       0.521795 -0.070702         0.888289   
1   0.031183          0.615102       0.521795  0.574118         0.888289   
2   0.031183          0.724515       0.521795  0.465563         0.888289   
3   0.031183          0.400194       0.521795  0.363926         0.888289   
4   0.031183          0.804349       0.521795  0.771447         0.888289   

   No Finding  
0         1.0  
1         0.5  
2         0.5  
3   

In [49]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
import joblib

features = ['NF_image', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
            'Pleural Effusion', 'Lung Opacity', 'Fracture']

# Split first
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Fit scaler on training data
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])

# Apply same scaler to val data
val_df[features] = scaler.transform(val_df[features])

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# Create datasets/loaders
train_dataset = RefinementDataset(train_df)
val_dataset = RefinementDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os

# Setup
model = NoFindingRefiner2().to(device)
criterion = nn.MSELoss()
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4) #1e-4
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)

# Save directory
os.makedirs("models", exist_ok=True)

# Early stopping
best_val_loss = float('inf')
patience_counter = 0
early_stopping_patience = 3

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        preds = model(x)
        #print(preds)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            loss = criterion(preds, y)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Val Loss: {avg_val_loss:.4f}")

    scheduler.step(avg_val_loss)

    # Early stopping and model saving
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        #print("✅ Saved new best model.")
    else:
        patience_counter += 1
        #print(f"⏳ No improvement. Patience: {patience_counter}/{early_stopping_patience}")

    if patience_counter >= early_stopping_patience:
        print("⛔ Early stopping triggered.")
        break

Epoch 1, Train Loss: 0.1446
Val Loss: 0.1215
Epoch 2, Train Loss: 0.1328
Val Loss: 0.1211
Epoch 3, Train Loss: 0.1299
Val Loss: 0.1208
Epoch 4, Train Loss: 0.1306
Val Loss: 0.1207
Epoch 5, Train Loss: 0.1299
Val Loss: 0.1207
Epoch 6, Train Loss: 0.1290
Val Loss: 0.1205
Epoch 7, Train Loss: 0.1297
Val Loss: 0.1204
Epoch 8, Train Loss: 0.1290
Val Loss: 0.1210
Epoch 9, Train Loss: 0.1293
Val Loss: 0.1213
Epoch 10, Train Loss: 0.1281
Val Loss: 0.1203


KeyboardInterrupt: 

In [17]:
feature_cols = ['NF_image', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                         'Pleural Effusion', 'Lung Opacity', 'Fracture']

for col in feature_cols:
    print(f"{col}: mean={df[col].mean():.4f}, std={df[col].std():.4f}")


NF_image: mean=-0.5124, std=0.5754
Enlarged Cardiomediastinum: mean=-0.3073, std=0.4978
Cardiomegaly: mean=-0.3126, std=0.6549
Pleural Effusion: mean=-0.2212, std=0.6929
Lung Opacity: mean=0.6031, std=0.4771
Fracture: mean=0.2377, std=0.4108


# Making the Dataset

In [15]:
df = pd.read_csv('train2023.csv')
df= df.dropna(subset=[target_col]).copy()
df[target_col] = (df[target_col] + 1) / 2

In [16]:
sample_sizes = {
    0.0: 10000,
    0.5: 6000,
    1.0: 10000
}

dfs = []
for label, n_samples in sample_sizes.items():
    class_df = df[df[target_col] == label]
    # Sample with replacement if not enough data
    sampled_df = class_df.sample(n=n_samples, replace=(n_samples > len(class_df)), random_state=42)
    dfs.append(sampled_df)

# Combine and shuffle
balanced_df = pd.concat(dfs).sample(frac=1, random_state=42).reset_index(drop=True)
df = balanced_df
valid_ids = set(df['Unnamed: 0'].astype(str))  # or 'Id'

In [17]:
label_counts = df[target_col].value_counts()
print(label_counts)

No Finding
0.0    10000
1.0    10000
0.5     6000
Name: count, dtype: int64


In [18]:
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes, hidden_dim = 512, dropout_rate = 0.5):
        super(MultiLabelResNet50, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, hidden_dim),  # New intermediate layer. ##512 --> 256
            nn.ReLU(),
            nn.Dropout(dropout_rate),  # Dropout to prevent overfitting ##0.5 --> 0.6
            nn.Linear(hidden_dim, num_classes),  # Output layer
            nn.Sigmoid()  # Sigmoid for multi-label classification (soften the data)
            #nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [19]:
class MultiLabelResNet50_2(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelResNet50_2, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 256),  # New intermediate layer. ##512 --> 256
            nn.ReLU(),
            nn.Dropout(0.6),  # Dropout to prevent overfitting ##0.5 --> 0.6
            nn.Linear(256, num_classes),  # Output layer
            nn.Sigmoid()  # Sigmoid for multi-label classification (soften the data)
            #nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [20]:
model_pel = MultiLabelResNet50(num_classes=1).to(device)
model_pel.load_state_dict(torch.load('models/pe_lat_model.pth'))
model_pel.eval()

model_pef = MultiLabelResNet50(num_classes=1).to(device)
model_pef.load_state_dict(torch.load('models/pe_front_model.pth'))
model_pef.eval()

model_cml = MultiLabelResNet50_2(num_classes=1).to(device)
model_cml.load_state_dict(torch.load('models/cm_lat_model2.pth'))
model_cml.eval()

model_cmf = MultiLabelResNet50(num_classes=1).to(device)
model_cmf.load_state_dict(torch.load('models/cm_front_model.pth'))
model_cmf.eval()

model_lo = MultiLabelResNet50(num_classes=1, hidden_dim=256).to(device)
model_lo.load_state_dict(torch.load('amb_models/lo_partial/epoch_3.pth'))
model_lo.eval()

model_fr = MultiLabelResNet50_2(num_classes=1).to(device)
model_fr.load_state_dict(torch.load('models/best_fr4_model.pth'))
model_fr.eval()

model_nf = MultiLabelResNet50(num_classes=1, hidden_dim=256).to(device)
model_nf.load_state_dict(torch.load('amb_models/nf_partial/epoch_3.pth'))
model_nf.eval()

model_ec= MultiLabelResNet50(num_classes=1).to(device)
model_ec.load_state_dict(torch.load('models/best_ec1_model.pth'))
model_ec.eval()

MultiLabelResNet50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [21]:
import torch
import os
import pandas as pd
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
dir_dict = {
    "frontal": "input_images/train_frontal",  ##NOTE THIS CHANGED TO SOLUTION IMAGES
    "lateral": "input_images/train_lateral"
}

# Column setup
columns = ["Id", "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", 
           "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"]

average_values = {
    "No Finding": -0.734655,
    "Enlarged Cardiomediastinum": -0.275805,
    "Cardiomegaly": 0.190770,
    "Lung Opacity": 0.836288,
    "Pneumonia": 0.031183,
    "Pleural Other": 0.521795,
    "Fracture": 0.392374,
    "Support Devices": 0.888289
}

batch_size = 64
predictions = []

# Loop over both frontal and lateral directories
for view_type, test_dir in dir_dict.items():
    model_pe = model_pef if view_type == "frontal" else model_pel
    model_cm = model_cmf if view_type == "frontal" else model_cml
   # model_nf = model_nff if view_type == "frontal" else model_nfl
    file_list = [f for f in os.listdir(test_dir)
             if f.endswith(".pt") and f.split('.')[0] in valid_ids]

    batch = []
    batch_filenames = []

    for filename in tqdm(file_list, desc=f"Processing {view_type}"):
        image_path = os.path.join(test_dir, filename)
        image_tensor = torch.load(image_path).to(device)
        batch.append(image_tensor)
        batch_filenames.append(filename.split('.')[0])

        if len(batch) == batch_size or filename == file_list[-1]:
            input_batch = torch.stack(batch)

            with torch.no_grad():
                output_pe = model_pe(input_batch).cpu().numpy()
                output_cm = model_cm(input_batch).cpu().numpy()
                output_fr = model_fr(input_batch).cpu().numpy()
                output_ec = model_ec(input_batch).cpu().numpy()
                output_nf = model_nf(input_batch).cpu().numpy()
                output_lo = model_lo(input_batch).cpu().numpy()

            for i in range(len(batch)):
                pe_score = output_pe[i][0] * 2 - 1
                cm_score = output_cm[i][0] * 2 - 1
                fr_score = output_fr[i][0] * 2 - 1
                ec_score = output_ec[i][0] * 2 - 1
                nf_score = output_nf[i][0] * 2 - 1
                lo_score = output_lo[i][0] * 2 - 1

                row = [batch_filenames[i]]
                for col in columns[1:]:
                    if col == "Pleural Effusion":
                        row.append(pe_score)
                    elif col == "Cardiomegaly":
                        row.append(cm_score)
                    elif col == "Fracture":
                        row.append(fr_score)
                    elif col == "Enlarged Cardiomediastinum":
                        row.append(ec_score)
                    elif col == "No Finding":
                        row.append(nf_score)
                    elif col == "Lung Opacity":
                        row.append(lo_score)
                    else:
                        row.append(average_values.get(col, 0))

                predictions.append(row)

            batch = []
            batch_filenames = []

# Save all predictions
df_predictions = pd.DataFrame(predictions, columns=columns)
df_predictions = df_predictions.sort_values(by="Id")
df_predictions.to_csv('train_predictions.csv', index=False)

print("✅ Predictions saved to 'train_predictions.csv'")

Processing lateral: 100%|██████████| 5458/5458 [03:07<00:00, 29.03it/s]


✅ Predictions saved to 'train_predictions.csv'
